##### pysam docs

https://pysam.readthedocs.io/en/latest/index.html

In [1]:
import numpy as np
import pysam

### Grabbing the SAM file and grabbing all the seqs

## Need to set some variables here:

    -- SAM files
    -- freqs
    -- ref names
    -- % AR to simulate

In [15]:
samfile_A = pysam.AlignmentFile("pol_a.sam", "r")
samfile_B = pysam.AlignmentFile("pol_b.sam", "r")

# all_reads_A = samfile_A.fetch()
# all_reads_B = samfile_B.fetch()

# read_A = next(all_reads_A)
# read_B = next(all_reads_B)

master = {.55: ("pol_a", samfile_A), .45: ("pol_b", samfile_B)}
f = list(master.keys())
freqs = np.array(f)

## this is just to test 
reads_a = master[.45][1]
reads_b = master[.55][1]
# read = next(reads)
# print(read)

for i in range(3):
    new_read_a = []
    new_read_b = []
    for i in range(4):
        x_a = next(reads_a)
        y_b = next(reads_b)
        new_read_a.append(x_a)
        new_read_b.append(y_b)
    
    print(new_read_a[-1],"\n", new_read_b[-1],"\n", "\n")



B.US.2008.HIV_US_BID-V4123_2008.JQ403034-9997	0	0	450	99	120=	-1	-1	120	CCTGAAAATCCATACAATACTCCAGTATTTGCCATCAAGAAAAAAGACAGTACTAAATGGAGAAAATTAGTAGATTTCAGAGAACTTAATAAAAGAACTCAAGACTTCTGGGAAGTTCAA	array('B', [32, 34, 32, 33, 34, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 37, 38, 38, 38, 34, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 16, 38, 38, 38, 38, 38, 38, 38, 38, 25, 38, 36, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 33, 38, 38, 38, 37, 37, 36, 38, 37, 38, 38, 38, 33, 36, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 37, 38, 13, 38, 28, 38, 38, 38, 38, 38, 38, 37])	[] 
 B.CH.2002.HIV_CH_BID-V3527_2002.JQ403021-9997	0	0	450	99	120=	-1	-1	120	CCTGAAAATCCATACAATACTCCAATATTTGCTATAAAGAAAAAAGACAGTACTAAATGGAGAAAATTAGTAGATTTCAGAGAACTTAATAAGAAAACTCAAGACTTCTGGGAAGTTCAG	array('B', [32, 34, 32, 33, 34, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 37, 38, 38, 38, 34, 38, 38, 38, 38, 38, 38, 

In [3]:
f = list(master.keys())
freqs = np.array(f)

for q in range(2):    
    try:
        print("\n")
        i = np.random.choice(2, 1, True, freqs)[0]
        original_ref = f[i]

        ## this will be one of the refs
        print(original_ref)
        ref_1_name, reads_a = master[original_ref][0], master[original_ref][1]
        print("We will grab a read from", ref_1_name, "to Artificially Recombine")

        ## do we always want to just grab the next, or jump ahead, jump behind, etc ##
        #seq_getter = np.random.randint(len(list(reads_1)))
        
        read_a = next(reads_a)  
        print(read_a.qname)
        
        ## re order the list based on the freqs
        freq_reweight_list = freqs[np.arange(len(freqs)) !=i]
        freq_reweight = freq_reweight_list / np.sum(freq_reweight_list)
        j = np.random.choice(len(freq_reweight), 1, True, freq_reweight)[0]


        ## now grab the info from the other ref ## 
        ref_to_select = freq_reweight_list[j]
        print("selecting --> ", ref_to_select)
        ref_2_name, reads_b = master[ref_to_select][0], master[ref_to_select][1]
        print("We will now draw a read from: ", ref_2_name, " to Artificially Recombine with ", ref_1_name)

        print(reads_b)
        print(read_a.reference_start, read_a.reference_end)
        
        
        good_reads = list(filter(
        lambda read: read.reference_start == read_a.reference_start and read.reference_end == read_a.reference_end,
        reads_b
        ))

        print(good_reads)
        
    
    except StopIteration:
        pass

    

    '''
    if len(good_reads) == 0:
        continue
    else:
        seq_read_2 = good_reads[0].seq
        seq_read_1 = read_1.seq
    
        print("1 --> ",seq_read_1)
        print("2 --> ",seq_red_2)
        grabber = np.random.randint(len(seq_red_2))
        print("random grabber --> ", grabber)

        indexed_seq_1_start = seq_read_1[:grabber]
        indexed_seq_2_end = seq_read_2[grabber:]

        print("total len (should be len of read) -->", len(indexed_seq_2_end) + len(indexed_seq_1_start))
        print("len of 1--> ",len(indexed_seq_1_start))
        print("len of 2 -->",len(indexed_seq_2_end))
        print("\n")
    '''



0.45
We will grab a read from pol_b to Artificially Recombine
B.US.2008.HIV_US_BID-V4123_2008.JQ403034-9997
selecting -->  0.55
We will now draw a read from:  pol_a  to Artificially Recombine with  pol_b
450 570
[<pysam.libcalignedsegment.AlignedSegment object at 0x11b442440>, <pysam.libcalignedsegment.AlignedSegment object at 0x11b442590>, <pysam.libcalignedsegment.AlignedSegment object at 0x11b442830>, <pysam.libcalignedsegment.AlignedSegment object at 0x11b442670>]


0.55
We will grab a read from pol_a to Artificially Recombine
